In [1]:
import shutil
import subprocess
import sys
import tempfile
from pathlib import Path

import autochem

# User inputs:
formula = "C5H7"
Hf = 86.239
Tf = 0
units = {"energy": "kcal"}
run_path = "run"
# run_path = None

In [2]:
# 0. Create temporary directory to run in
tmp_dir = None if run_path else tempfile.TemporaryDirectory(dir=".")
run_path = Path(run_path or tmp_dir.name)
run_path.mkdir(exist_ok=True)
pf_inp_name = Path("pf.inp")
pf_inp_path = shutil.copyfile(pf_inp_name, run_path / pf_inp_name)

In [3]:
# 1. Run MESS-PF and read output to string
subprocess.run(["messpf", pf_inp_name], cwd=run_path)
pf_str = pf_inp_path.with_suffix(".dat").read_text()

In [4]:
# 2. Initialize autochem.therm.Species object with partition function data
spc = autochem.therm.from_messpf_output_string(
    pf_str, formula=formula, Hf=Hf, Tf=Tf, units=units
)

In [5]:
# 3. Write PAC99 input and run PAC99
pac_inp_name = Path(formula).with_suffix(".i97")
pac_inp_path = run_path / pac_inp_name
pac_inp_path.write_text(autochem.therm.pac99_input_string(spc))
# Copy PAC99's new.groups data file
pac_share_path = Path(sys.prefix) / "share" / "pac99"
for pac_share_file in pac_share_path.iterdir():
    shutil.copyfile(pac_share_file, run_path / pac_share_file.name)
pac_log_path = run_path / pac_inp_name.with_suffix(".log")
subprocess.run(
    ["pac99"],
    input=pac_inp_name.stem.encode(),
    stdout=pac_log_path.open("w"),
    cwd=run_path,
)

Note: The following floating-point exceptions are signalling: IEEE_DIVIDE_BY_ZERO


CompletedProcess(args=['pac99'], returncode=0)

In [6]:
# 4. Read PAC99 output and convert to Chemkin
pac_out_name = pac_inp_name.with_suffix(".c97")
pac_out_path = run_path / pac_out_name
pac_out_str = pac_out_path.read_text()
spc = autochem.therm.from_pac99_output_string(pac_out_str)
ckin_str = autochem.therm.chemkin_string(spc)
ckin_name = pac_inp_name.with_suffix(".ckin")
ckin_path = Path(".") / ckin_name
ckin_path.write_text(ckin_str)
print(ckin_str)

C5H7(487)z              C   5H   7          G     200.0    3000.0  1000.0      1
 5.44139387E+00 2.99849337E-02-1.45175641E-05 3.42018649E-09-3.17917319E-13    2
 3.84863610E+04 1.50413709E+00 8.03819494E+00-9.15747006E-03 1.04299288E-04    3
-1.30879516E-07 5.17105360E-11 3.90243795E+04-4.94049720E+00                   4


In [7]:
if tmp_dir:
    tmp_dir.cleanup()